In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
#import spacy
import string
import itertools 
df = pd.read_csv("/content/drive/MyDrive/Social Media Analytics/Datasets for Practice/Product Sentiments.csv")

df.head()

,Sentiment,Time,Product,Username,Comment
0,Positive,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,Positive,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,Positive,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,Positive,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,Positive,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...


## Emotion Analysis

### Using NRC

In [ ]:
pip install NRCLex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from nrclex import NRCLex
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
text = "I love using transformers. The best part is wide range of support and its easy to use"
text_object = NRCLex(text)
print(text_object.top_emotions)
print(text_object.affect_list)
print(text_object.sentences)
print(text_object.words)
print(text_object.affect_dict)
print(text_object.raw_emotion_scores)
print(text_object.affect_frequencies)

[('positive', 0.5), ('joy', 0.5)]
['joy', 'positive']
[Sentence("I love using transformers."), Sentence("The best part is wide range of support and its easy to use")]
['I', 'love', 'using', 'transformers', 'The', 'best', 'part', 'is', 'wide', 'range', 'of', 'support', 'and', 'its', 'easy', 'to', 'use']
{'love': ['joy', 'positive']}
{'joy': 1, 'positive': 1}
{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 'trust': 0.0, 'surprise': 0.0, 'positive': 0.5, 'negative': 0.0, 'sadness': 0.0, 'disgust': 0.0, 'joy': 0.5}


In [ ]:
def nrc_emo(text):
  text_object = NRCLex(text)
  return text_object.affect_frequencies['fear'],text_object.affect_frequencies['anger'], \
  text_object.affect_frequencies['anticip'], text_object.affect_frequencies['trust'],\
  text_object.affect_frequencies['surprise'], text_object.affect_frequencies['sadness'], \
  text_object.affect_frequencies['disgust'], text_object.affect_frequencies['joy']


In [ ]:
df['fear'], df['anger'],df['anticip'],	df['trust'],\
df['surprise'],df['sadness'],df['disgust'],df['joy'] = zip(*df['Comment'].apply(lambda x: nrc_emo(x)))
df

,Sentiment,Time,Product,Username,Comment,fear,anger,anticip,trust,surprise,sadness,disgust,joy
0,Positive,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.00
1,Positive,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...,0.000000,0.000000,0.0,0.20,0.2,0.000000,0.000000,0.20
2,Positive,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck...",0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.00
3,Positive,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...,0.000000,0.000000,0.0,0.00,0.0,0.250000,0.000000,0.25
4,Positive,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...,0.000000,0.000000,0.0,0.25,0.0,0.000000,0.000000,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,Neutral,Sun Jun 14 04:31:43 UTC 2009,latex,proggit,Ask Programming: LaTeX or InDesign?: submitted...,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.00
494,Negative,Sun Jun 14 04:32:17 UTC 2009,latex,sam33r,"On that note, I hate Word. I hate Pages. I hat...",0.217391,0.173913,0.0,0.00,0.0,0.217391,0.173913,0.00
495,Positive,Sun Jun 14 04:36:34 UTC 2009,latex,iamtheonlyjosie,Ahhh... back in a *real* text editing environm...,0.000000,0.000000,0.0,0.50,0.0,0.000000,0.000000,0.00
496,Negative,Sun Jun 14 21:36:07 UTC 2009,iran,plutopup7,"Trouble in Iran, I see. Hmm. Iran. Iran so far...",0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.00


In [ ]:
df['nrc_emotion'] = df[['fear','anger','anticip','trust','surprise','sadness','disgust','joy']].idxmax(axis=1)
df

,Sentiment,Time,Product,Username,Comment,fear,anger,anticip,trust,surprise,sadness,disgust,joy,nrc_emotion
0,Positive,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.00,fear
1,Positive,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...,0.000000,0.000000,0.0,0.20,0.2,0.000000,0.000000,0.20,trust
2,Positive,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck...",0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.00,fear
3,Positive,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...,0.000000,0.000000,0.0,0.00,0.0,0.250000,0.000000,0.25,sadness
4,Positive,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...,0.000000,0.000000,0.0,0.25,0.0,0.000000,0.000000,0.25,trust
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,Neutral,Sun Jun 14 04:31:43 UTC 2009,latex,proggit,Ask Programming: LaTeX or InDesign?: submitted...,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.00,fear
494,Negative,Sun Jun 14 04:32:17 UTC 2009,latex,sam33r,"On that note, I hate Word. I hate Pages. I hat...",0.217391,0.173913,0.0,0.00,0.0,0.217391,0.173913,0.00,fear
495,Positive,Sun Jun 14 04:36:34 UTC 2009,latex,iamtheonlyjosie,Ahhh... back in a *real* text editing environm...,0.000000,0.000000,0.0,0.50,0.0,0.000000,0.000000,0.00,trust
496,Negative,Sun Jun 14 21:36:07 UTC 2009,iran,plutopup7,"Trouble in Iran, I see. Hmm. Iran. Iran so far...",0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.00,fear


In [ ]:
df['nrc_emotion'].value_counts()

fear        328
trust        95
anger        25
joy          20
surprise     13
sadness      10
disgust       7
Name: nrc_emotion, dtype: int64

### Using Bert Emotion

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)

In [ ]:
text = 'I liked the product very much.'
prediction = classifier(text)
prediction

[[{'label': 'joy', 'score': 0.9841485619544983},
  {'label': 'neutral', 'score': 0.005507329013198614},
  {'label': 'disgust', 'score': 0.0035264489706605673},
  {'label': 'surprise', 'score': 0.002149660838767886},
  {'label': 'anger', 'score': 0.0021169278770685196},
  {'label': 'sadness', 'score': 0.0020234023686498404},
  {'label': 'fear', 'score': 0.0005276770098134875}]]

In [ ]:
def bert_emo(text):
  prediction = classifier(text)
  flattenlist = list(itertools.chain(*prediction))
  data = pd.DataFrame.from_dict(flattenlist)
  data.sort_values(by = 'label', axis = 0, inplace = True)
  score = data['score'].values
  return score

In [ ]:
df['bert_emo_anger'],df['bert_emo_disgust'],df['bert_emo_fear'], df['bert_emo_joy'],df['bert_emo_neutral'],\
df['bert_emo_sadness'], df['bert_emo_surprise'] =zip(*df['Comment'].apply(lambda x: bert_emo(str(x))))
df

,Sentiment,Time,Product,Username,Comment,fear,anger,anticip,trust,surprise,...,disgust,joy,nrc_emotion,bert_emo_anger,bert_emo_disgust,bert_emo_fear,bert_emo_joy,bert_emo_neutral,bert_emo_sadness,bert_emo_surprise
0,Positive,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...,0.000000,0.000000,0.0,0.00,0.0,...,0.000000,0.00,fear,0.003191,0.000873,0.001479,0.699805,0.057349,0.070188,0.167115
1,Positive,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...,0.000000,0.000000,0.0,0.20,0.2,...,0.000000,0.20,trust,0.002590,0.001151,0.000639,0.769643,0.090562,0.030881,0.104534
2,Positive,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck...",0.000000,0.000000,0.0,0.00,0.0,...,0.000000,0.00,fear,0.788000,0.012412,0.023202,0.129594,0.020804,0.019808,0.006181
3,Positive,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...,0.000000,0.000000,0.0,0.00,0.0,...,0.000000,0.25,sadness,0.008011,0.001440,0.002054,0.170188,0.106894,0.595466,0.115947
4,Positive,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...,0.000000,0.000000,0.0,0.25,0.0,...,0.000000,0.25,trust,0.005677,0.000682,0.003537,0.194958,0.074210,0.028921,0.692015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,Neutral,Sun Jun 14 04:31:43 UTC 2009,latex,proggit,Ask Programming: LaTeX or InDesign?: submitted...,0.000000,0.000000,0.0,0.00,0.0,...,0.000000,0.00,fear,0.004367,0.001354,0.003311,0.009749,0.864873,0.004701,0.111645
494,Negative,Sun Jun 14 04:32:17 UTC 2009,latex,sam33r,"On that note, I hate Word. I hate Pages. I hat...",0.217391,0.173913,0.0,0.00,0.0,...,0.173913,0.00,fear,0.603009,0.342011,0.007843,0.001860,0.026541,0.016328,0.002409
495,Positive,Sun Jun 14 04:36:34 UTC 2009,latex,iamtheonlyjosie,Ahhh... back in a *real* text editing environm...,0.000000,0.000000,0.0,0.50,0.0,...,0.000000,0.00,trust,0.006849,0.015375,0.004976,0.038905,0.409417,0.007926,0.516553
496,Negative,Sun Jun 14 21:36:07 UTC 2009,iran,plutopup7,"Trouble in Iran, I see. Hmm. Iran. Iran so far...",0.000000,0.000000,0.0,0.00,0.0,...,0.000000,0.00,fear,0.011012,0.000360,0.956373,0.003266,0.001609,0.025670,0.001711


In [ ]:
df['bert_emotion'] = df[['bert_emo_anger','bert_emo_disgust','bert_emo_fear','bert_emo_joy',\
                         'bert_emo_neutral','bert_emo_sadness','bert_emo_surprise']].idxmax(axis=1)
df

,Sentiment,Time,Product,Username,Comment,fear,anger,anticip,trust,surprise,...,joy,nrc_emotion,bert_emo_anger,bert_emo_disgust,bert_emo_fear,bert_emo_joy,bert_emo_neutral,bert_emo_sadness,bert_emo_surprise,bert_emotion
0,Positive,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...,0.000000,0.000000,0.0,0.00,0.0,...,0.00,fear,0.003191,0.000873,0.001479,0.699805,0.057349,0.070188,0.167115,bert_emo_joy
1,Positive,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...,0.000000,0.000000,0.0,0.20,0.2,...,0.20,trust,0.002590,0.001151,0.000639,0.769643,0.090562,0.030881,0.104534,bert_emo_joy
2,Positive,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck...",0.000000,0.000000,0.0,0.00,0.0,...,0.00,fear,0.788000,0.012412,0.023202,0.129594,0.020804,0.019808,0.006181,bert_emo_anger
3,Positive,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...,0.000000,0.000000,0.0,0.00,0.0,...,0.25,sadness,0.008011,0.001440,0.002054,0.170188,0.106894,0.595466,0.115947,bert_emo_sadness
4,Positive,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...,0.000000,0.000000,0.0,0.25,0.0,...,0.25,trust,0.005677,0.000682,0.003537,0.194958,0.074210,0.028921,0.692015,bert_emo_surprise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,Neutral,Sun Jun 14 04:31:43 UTC 2009,latex,proggit,Ask Programming: LaTeX or InDesign?: submitted...,0.000000,0.000000,0.0,0.00,0.0,...,0.00,fear,0.004367,0.001354,0.003311,0.009749,0.864873,0.004701,0.111645,bert_emo_neutral
494,Negative,Sun Jun 14 04:32:17 UTC 2009,latex,sam33r,"On that note, I hate Word. I hate Pages. I hat...",0.217391,0.173913,0.0,0.00,0.0,...,0.00,fear,0.603009,0.342011,0.007843,0.001860,0.026541,0.016328,0.002409,bert_emo_anger
495,Positive,Sun Jun 14 04:36:34 UTC 2009,latex,iamtheonlyjosie,Ahhh... back in a *real* text editing environm...,0.000000,0.000000,0.0,0.50,0.0,...,0.00,trust,0.006849,0.015375,0.004976,0.038905,0.409417,0.007926,0.516553,bert_emo_surprise
496,Negative,Sun Jun 14 21:36:07 UTC 2009,iran,plutopup7,"Trouble in Iran, I see. Hmm. Iran. Iran so far...",0.000000,0.000000,0.0,0.00,0.0,...,0.00,fear,0.011012,0.000360,0.956373,0.003266,0.001609,0.025670,0.001711,bert_emo_fear


In [ ]:
df['bert_emotion'].value_counts()

bert_emo_neutral     147
bert_emo_joy         114
bert_emo_surprise     81
bert_emo_sadness      70
bert_emo_anger        38
bert_emo_fear         25
bert_emo_disgust      23
Name: bert_emotion, dtype: int64